# User Item Biases With Regularization
* This alpha is equivalent to the explicit version, except 
  we use cross-entropy loss insead of mean squared error
* See UserItemBiases.ipynb for more details

In [ ]:
const name = "UserItemBiases"
const residual_alphas = [];

In [ ]:
using NBInclude
@nbinclude("Alpha.ipynb");

In [ ]:
const training = get_residuals("training", residual_alphas)
const validation = get_residuals("validation", residual_alphas);

## Training
* There is no closed form solution, so we optimize the loss function numerically

In [ ]:
# TODO thread this
# TODO wrap inside a memoized function
# TODO. try in ItemCF
user_to_idxs = [[] for x = 1:maximum(training.user)];
@showprogress for i = 1:length(training.user)
    push!(user_to_idxs[training.user[i]], i)
end

In [ ]:
function make_prediction(u, a)
    p = exp.(mean(u) .+ a)
    p = p / sum(p)
    p
end;

In [ ]:
function make_prediction(users, items, u, a)
    p = zeros(eltype(u), length(users))
    @tprogress Threads.@threads for user in collect(Set(users))
        mask = user_to_idxs[user]
        p[mask] .= make_prediction(u[user], a)[items[mask]]
    end
    @info "A"
    p
end;

In [ ]:
const u = convert.(Float32, randn(maximum(training.user)))
const a = convert.(Float32, randn(maximum(training.item)));

In [ ]:
p = make_prediction(training.user, training.item, u, a)

In [ ]:
p[1:10]

In [ ]:
function solve(training, weights, λ_u, λ_a)
    users, items, ratings = training.user, training.item, training.rating    
    u = zeros(eltype(λ_u), maximum(users))
    a = zeros(eltype(λ_a), maximum(items))
    u, a
end

In [ ]:
function train_model(training, stop_criteria, λ_u, λ_a, λ_wu, λ_wa)
    @info "training model with parameters [$λ_u, $λ_a, $λ_wu, $λ_wa]"
    weights =
        safe_exp.(get_counts("training"), log(λ_wu)) .*
        safe_exp.(get_counts("training"; by_item = true), log(λ_wa))
    solve(training, weights, λ_u, λ_a)
end;

## Training

In [ ]:
function validation_mse(λ)
    λ = exp.(λ) # ensure λ is nonnegative
    stop_criteria = convergence_stopper(1e-6, max_iters = 16)
    u, a = train_model(training, stop_criteria, λ...)
    pred_score = make_prediction(validation.user, validation.item, u, a)
    weights = get_weights("validation", "inverse")
    mse(validation.rating, pred_score, weights)
end;

In [ ]:
# Find the best regularization hyperparameters
res = optimize(
    validation_mse,
    fill(0.0f0, 4),
    LBFGS(),
    autodiff = :forward,
    Optim.Options(show_trace = true, extended_trace = true),
);
λ = exp.(Optim.minimizer(res));

In [ ]:
@info "The optimal λ is $λ, found in " * repr(Optim.f_calls(res)) * " function calls"

In [ ]:
stop_criteria = convergence_stopper(1e-6, max_iters = 16)
u, a = train_model(training, stop_criteria, λ...);

## Inference

In [ ]:
model(users, items) = make_prediction(users, items, u, a);

In [ ]:
write_predictions(model; residual_alphas = residual_alphas);

In [ ]:
write_params(Dict("u" => u, "a" => a, "λ" => λ));